In [15]:
import json
import sqlite3
import pandas as pd


In [16]:
jsonData = open('RawData/20181120_01.json').read()

In [17]:
rawData = json.loads(jsonData)

In [18]:
leaders = {'Gilgamesh':1, 'Montezuma':2 , 'Bandar Brunei':3, 'Babylon':4, 'Hong Kong':5 }
print(leaders['Gilgamesh'])
cities = {'Uruk':1, 'Tenochtitlan':2, 'Bandar Brunei':3, 'Babylon':4, 'Hong Kong':5 }
print(cities['Uruk'])

1
1


In [60]:
cnx = sqlite3.connect('Database/Civ6CitySettledData.db')
cur = cnx.cursor()
print(cnx)
print(cur)

# Played with sqlalchemy, but not particularly useful when using sqlite
#engine = create_engine('sqlite:///Database/Civ6CitySettledData.db')

In [61]:
sqlSelect = 'SELECT * FROM citySettled'
citiesDf = pd.read_sql_query(sqlSelect, cnx)
print(citiesDf)

sqlSelect = 'SELECT * FROM leader'
leadersDf = pd.read_sql_query(sqlSelect, cnx)
print(leadersDf)


   cityId           name  settledBy  onTurn
0       1           Uruk          1       1
1       2   Tenochtitlan          2       1
2       3  Bandar Brunei          3       1
3       4        Babylon          4       1
4       5      Hong Kong          5       1
   leaderId           name            civ isMajor
0         1      Gilgamesh        Sumeria    True
1         2      Montezuma          Aztec    True
2         3  Bandar Brunei  Bandar Brunei   False
3         4        Babylon        Babylon   False
4         5      Hong Kong      Hong Kong   False


In [42]:
# progressively narrow down
x = citiesDf[citiesDf.name == 'Babylon']
print(type(x),"\n", x)
x = x.iloc[0]
print("\n", type(x),"\n", x)
x = int(x.cityId)
print("\n", type(x),"\n", x)

<class 'pandas.core.frame.DataFrame'> 
    cityId     name  settledBy  onTurn
3       4  Babylon          4       1

 <class 'pandas.core.series.Series'> 
 cityId             4
name         Babylon
settledBy          4
onTurn             1
Name: 3, dtype: object

 <class 'int'> 
 4


In [62]:
def getCityId(cityName):
    try:
        # The iloc is neccessary as we can't assume there is only one return value!?
        # also have to make sure it is int as numpy.int64 doesn't go into sqlite so well?
        return int(citiesDf[citiesDf.name == cityName].iloc[0].cityId)
    except IndexError:
        return None
    
def getLeaderId(ownerName, ownerCiv):
    try:
        # The iloc is necessary as we can't assume there is only one return value!?
        # also have to make sure it is int as numpy.int64 doesn't go into sqlite so well?
        return int(leadersDf[leadersDf.name == ownerName].iloc[0].leaderId)
    except IndexError:
        return None

In [45]:
# building up the cityPlotsSettled data for entry...
result = [] #{}
#i = 0
for city in rawData['cityOnSettledLog']:
    cityPlotInfo = city['cityPlotInfo']
    plotInfo = {
        'plotId': None,
        'ownerCityId': getCityId(cityPlotInfo['ownerCity']),
        'ring': 0,
        'idx': cityPlotInfo['index'],
        'terrain': cityPlotInfo['terrain'],
        'feature': cityPlotInfo['feature'],
        'resource': cityPlotInfo['resource'],
        'resourceCount': cityPlotInfo['resourceCount'],
        'resourceType': cityPlotInfo['resourceType'],
        'workers': cityPlotInfo['workers'],
        'district': cityPlotInfo['district'],
        'hasRiver': cityPlotInfo['hasRiver'],
        'isWater': cityPlotInfo['isWater'],
        'isLake': cityPlotInfo['isLake'],
        'isCity': cityPlotInfo['isCity'],
    }
 #   result[i] = plotInfo
 #   i = i + 1
    result.append(plotInfo)
    for plot in city['ring1PlotInfo']:
        plotInfo = {
            'plotId': None,
            'ownerCityId': getCityId(plot['ownerCity']),
            'ring': 1,
            'idx': plot['index'],
            'terrain': plot['terrain'],
            'feature': plot['feature'],
            'resource': plot['resource'],
            'resourceCount': plot['resourceCount'],
            'resourceType': plot['resourceType'],
            'workers': plot['workers'],
            'district': plot['district'],
            'hasRiver': plot['hasRiver'],
            'isWater': plot['isWater'],
            'isLake': plot['isLake'],
            'isCity': plot['isCity'],
        }
        #result[i] = plotInfo
        #i = i + 1
        result.append(plotInfo)
    for plot in city['ring2PlotInfo']:
        plotInfo = {
            'plotId': None,
            'ownerCityId': getCityId(plot['ownerCity']),
            'ring': 2,
            'idx': plot['index'],
            'terrain': plot['terrain'],
            'feature': plot['feature'],
            'resource': plot['resource'],
            'resourceCount': plot['resourceCount'],
            'resourceType': plot['resourceType'],
            'workers': plot['workers'],
            'district': plot['district'],
            'hasRiver': plot['hasRiver'],
            'isWater': plot['isWater'],
            'isLake': plot['isLake'],
            'isCity': plot['isCity'],
        }
        #result[i] = plotInfo
        #i = i + 1
        result.append(plotInfo)


In [46]:
print(result[0])
print(result[1])
print(result[2])

{'plotId': None, 'ownerCityId': 1, 'ring': 0, 'idx': 760, 'terrain': 'Plains (Hills)', 'feature': 'None', 'resource': 'None', 'resourceCount': 0, 'resourceType': 'None', 'workers': 1, 'district': 'City Center', 'hasRiver': True, 'isWater': False, 'isLake': False, 'isCity': True}
{'plotId': None, 'ownerCityId': 1, 'ring': 1, 'idx': 805, 'terrain': 'Plains (Mountain)', 'feature': 'None', 'resource': 'None', 'resourceCount': 0, 'resourceType': 'None', 'workers': 0, 'district': 'None', 'hasRiver': False, 'isWater': False, 'isLake': False, 'isCity': False}
{'plotId': None, 'ownerCityId': 1, 'ring': 1, 'idx': 761, 'terrain': 'Plains (Hills)', 'feature': 'None', 'resource': 'Sheep', 'resourceCount': 1, 'resourceType': 'Bonus', 'workers': 0, 'district': 'None', 'hasRiver': False, 'isWater': False, 'isLake': False, 'isCity': False}


In [32]:
df = pd.DataFrame(result)
print(df.shape)
print(df.iloc[0])
print(df.iloc[0].to_dict())

(95, 15)
district            City Center
feature                    None
hasRiver                   True
idx                         760
isCity                     True
isLake                    False
isWater                   False
ownerCityId                   1
plotId                     None
resource                   None
resourceCount                 0
resourceType               None
ring                          0
terrain          Plains (Hills)
workers                       1
Name: 0, dtype: object
{'district': 'City Center', 'feature': 'None', 'hasRiver': True, 'idx': 760, 'isCity': True, 'isLake': False, 'isWater': False, 'ownerCityId': 1.0, 'plotId': None, 'resource': 'None', 'resourceCount': 0, 'resourceType': 'None', 'ring': 0, 'terrain': 'Plains (Hills)', 'workers': 1}


In [48]:
for res in result:
    cur.execute('INSERT INTO cityPlotsSettled VALUES(:plotId, :ownerCityId, :ring, :idx,:terrain, :feature, :resource, :resourceCount, :resourceType, :workers, :district, :hasRiver, :isWater, :isLake, :isCity )',
               res)


In [49]:
cnx.commit()
#for row in rawData['gameConfig'][0]['players']:
#    print(row['playerName'], row['playerCiv'])

In [67]:
cnx.close()

In [63]:
# building up the cityPerTurnLog data for entry...
result = []
for cityPerTurnEntry in rawData['cityPerTurnLog']:
    turn = int(cityPerTurnEntry['atEndTurn'])
    for cityEntry in cityPerTurnEntry['cityLog']:
        turnInfo = {
            'cityId': getCityId(cityEntry['cityName']),
            'turn': turn,
            'ownerId': getLeaderId(cityEntry['ownerName'], 'Blah'),
            'foodPerTurn': cityEntry['foodPerTurn'],
            'foodToolTip': cityEntry['foodToolTip'],
            'productionPerTurn': cityEntry['productionPerTurn'],
            'productionToolTip': cityEntry['productionToolTip'],
            'goldPerTurn': cityEntry['goldPerTurn'],
            'goldToolTip': cityEntry['goldToolTip'],
            'sciencePerTurn': cityEntry['sciencePerTurn'],
            'scienceToolTip': cityEntry['scienceToolTip'],
            'culturePerTurn': cityEntry['culturePerTurn'],
            'cultureToolTip': cityEntry['cultureToolTip'],
            'faithPerTurn': cityEntry['faithPerTurn'],
            'faithToolTip': cityEntry['faithToolTip'],
            'population': cityEntry['population'],
            'housing': cityEntry['housing'],
            'amenities': cityEntry['amenities'],
            'amenitiesNeeded': cityEntry['amenitiesNeeded'],
            'happiness': cityEntry['happiness'],
        }
        result.append(turnInfo)

print(result[0])
print(result[1])
print(result[2])

{'cityId': 1, 'turn': 1, 'ownerId': 1, 'foodPerTurn': '4.00', 'foodToolTip': '+4 from Worked Tiles', 'productionPerTurn': '6.30', 'productionToolTip': '+4 from Worked Tiles[NEWLINE]+2 from Buildings[NEWLINE]   +2 from Palace[NEWLINE]+5pct (+0.2) from Amenities', 'goldPerTurn': '5.25', 'goldToolTip': '+5 from Buildings[NEWLINE]   +5 from Palace[NEWLINE]+5pct (+0.2) from Amenities', 'sciencePerTurn': '2.63', 'scienceToolTip': '+2 from Buildings[NEWLINE]   +2 from Palace[NEWLINE]+0.5 from Population[NEWLINE]+5pct (+0.1) from Amenities', 'culturePerTurn': '1.36', 'cultureToolTip': '+1 from Buildings[NEWLINE]   +1 from Palace[NEWLINE]+0.2 from Population[NEWLINE]+5pct (+0) from Amenities', 'faithPerTurn': '0.00', 'faithToolTip': '+5pct (+0) from Amenities', 'population': 1, 'housing': 6, 'amenities': 1, 'amenitiesNeeded': 0, 'happiness': 'Happy'}
{'cityId': 2, 'turn': 1, 'ownerId': 2, 'foodPerTurn': '4.00', 'foodToolTip': '+4 from Worked Tiles', 'productionPerTurn': '5.00', 'productionToolT

In [64]:
for res in result:
    cur.execute('INSERT INTO cityPerTurnLog VALUES(:cityId, :turn, :ownerId, :foodPerTurn, :foodToolTip, :productionPerTurn, :productionToolTip, :goldPerTurn, :goldToolTip, :sciencePerTurn, :scienceToolTip, :culturePerTurn, :cultureToolTip, :faithPerTurn, :faithToolTip, :population, :housing, :amenities, :amenitiesNeeded, :happiness)',
        res)


In [65]:
cnx.commit()